In [1]:
import numpy as np
import tensorflow as tf
from keras.layers import Dense
import pandas as pd
from keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
df = pd.read_csv('credit_data.csv')

In [3]:
x_data = np.array(df.iloc[:497, 0:6], dtype=np.float32)
y_data = np.array(df.iloc[:497, 6], dtype=np.float32).reshape(-1, 1)
x_pred = np.array(df.iloc[497:, 0:6], dtype=np.float32)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1)

In [4]:
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))\
            .repeat()\
            .batch(12)\
            .make_one_shot_iterator()\
            .get_next()
    return {'x': dataset[0]}, dataset[1]

In [7]:
def model_fn(features, labels, mode):
    nInput = 6      # input layer의 neuron 개수
    nHidden = 12    # hidden layer의 neuron 개수
    nOutput = 1     # output layer의 neuron 개수

    H1 = Dense(nHidden, activation='relu')(features['x'])
    predY = Dense(nOutput, activation='sigmoid')(H1)
    
    # 학습
    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.reduce_mean(binary_crossentropy(labels, predY))
        optimizer = tf.train.AdamOptimizer(0.01)
        global_step = tf.train.get_global_step()
        train = optimizer.minimize(loss, global_step)
        
        return tf.estimator.EstimatorSpec(
                mode = mode,
                train_op = train,
                loss = loss)
    
    # 평가
    elif mode == tf.estimator.ModeKeys.EVAL:
        pass
    
    # 예측
    elif mode == tf.estimator.ModeKeys.PREDICT:
        pass

In [8]:
estimator = tf.estimator.Estimator(model_fn = model_fn)
estimator.train(train_input_fn, steps = 100)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\seong\\AppData\\Local\\Temp\\tmpuqzq4yf1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000015AE8886EF0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Checkp

In [17]:
yHat = sess.run(predY, feed_dict={x: x_test})
accuracy = accuracy_score(y_test, np.round(yHat))
print("정확도 = %.4f" % accuracy)

정확도 = 0.8000


In [11]:
yPred = sess.run(predY, feed_dict={x: x_pred})
print(yPred)

[[0.8504145 ]
 [0.23027574]]
